In [1]:
import numpy as np                  # type: ignore
import napari                       # type: ignore     
import myfunctions as mf            # type: ignore

##### Reading 4D sequence

In [2]:
sequence = mf.read_4Dsequence('P28A_FT_H_Exp1', first_slice=100, last_slice=104+1)

threshold = mf.find_threshold(sequence)

threshold = 2

segmented_sequence = mf.segment4D(sequence, threshold, filtering3D=True, smallest_4Dvolume=100)

Sequence collected in 0.19 s

Finding threshold...
Threshold=2.12 found in 0.26 s



Backward propagation: 100%|██████████| 53/53 [00:02<00:00, 17.69it/s]


Filtering...
Filtering completed in 0.60 s



In [3]:
viewer = napari.Viewer()

images = [viewer.add_image(sequence[:, i, :, :], 
                           name=f'Slice {100+i}', 
                           opacity=0.4, 
                           visible=False) 
                           for i in range(sequence.shape[1])]

labels = [viewer.add_labels(segmented_sequence[:, i, :, :], 
                            name=f'Label {100+i}', 
                            blending='additive', 
                            opacity=0.8, 
                            visible=False) 
                            for i in range(sequence.shape[1])]

images[0].visible = True
labels[0].visible = True

##### Reading 3D sequence

In [4]:
sequence = mf.read_3Dsequence('P28A_FT_H_Exp1', time=150)

threshold = mf.find_threshold(sequence)

segmented_sequence = mf.segment3D(sequence, threshold)

Finding threshold...
Threshold=2.00 found in 0.20 s



In [5]:
viewer = napari.Viewer()
_ = viewer.add_image(sequence, opacity=0.4)
_ = viewer.add_labels(segmented_sequence, blending='additive', opacity=0.8)